<a href="https://colab.research.google.com/github/Disaster4255/PL_REPO_PY_Hao/blob/main/ProgrammingLanguage114_1_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#空拍與旅遊景點紀錄<br>
有空拍點位分頁與旅遊紀錄點分頁，可以輸入地點名稱、經緯度與備註或限航條件等資訊<br>
刷新後會顯示在地圖上<br>
最下方有選單可刪除之前錯誤或不需要的資料

In [1]:
!pip install folium

In [27]:
# 1. 匯入必要的函式庫
import folium
import pandas as pd
import gradio as gr
# import google.generativeai as genai # 範例中未使用，可註解或移除
from datetime import datetime as dt
from dateutil.tz import gettz

# Google Auth & Sheets 相關函式庫
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth import default

# 2. Google Colab 驗證和 Gspread 授權
# 執行此段代碼會跳出視窗要求您登入 Google 帳號並授權 Colab 存取 Google Drive 相關服務
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
except Exception as e:
    print(f"Google 認證或授權失敗: {e}")
    # 在非 Colab 環境下可能需要使用 service_account
    # 這裡假設是在 Colab 中運行
    pass

# 3. 設定參數
# 請將此 URL 替換為您實際的 Google Sheets 網址
SHEET_URL = "https://docs.google.com/spreadsheets/d/16QWyRWV_tvRVOyLi3sdAxf_ryZWVHrZZapoCx_k3R2w/edit?usp=sharing"
WORKSHEET_1_NAME = "空拍點位"
WORKSHEET_2_NAME = "旅遊紀錄點"
TIMEZONE = "Asia/Taipei"

# 4. 核心函式 - 讀取/寫入/刪除資料和生成地圖

def load_data(worksheet_name):
    """從 Google Sheets 載入資料並轉換為 DataFrame。"""
    try:
        gsheets = gc.open_by_url(SHEET_URL)
        worksheet = gsheets.worksheet(worksheet_name)
        sheets_data = worksheet.get_all_values()
        if not sheets_data or len(sheets_data) < 2:
            # 如果工作表為空或只有標題，則創建一個空的 DataFrame
            # 確保返回的 DataFrame 包含正確的欄位名稱
            cols = sheets_data[0] if sheets_data else (
                ['地點名稱', '經度', '緯度', '地點描述', '飛航限制'] if worksheet_name == WORKSHEET_1_NAME else
                ['地點名稱', '經度', '緯度', '個人備註']
            )
            return pd.DataFrame(columns=cols)

        df = pd.DataFrame(sheets_data[1:], columns=sheets_data[0])
        # 確保經度和緯度是數字類型
        for col in ['經度', '緯度']:
            if col in df.columns:
                # 使用 to_numeric 轉換，errors='coerce' 將無效值轉換為 NaN
                df[col] = pd.to_numeric(df[col], errors='coerce')
        # 移除經緯度為NaN的行，以便在地圖上正確繪製
        return df.dropna(subset=['經度', '緯度'])

    except Exception as e:
        print(f"載入 {worksheet_name} 工作表失敗: {e}")
        # 返回一個帶有預期標題的空 DataFrame
        if worksheet_name == WORKSHEET_1_NAME:
            cols = ['地點名稱', '經度', '緯度', '地點描述', '飛航限制']
        else:
            cols = ['地點名稱', '經度', '緯度', '個人備註']
        return pd.DataFrame(columns=cols)

def write_data(worksheet_name, new_row_data):
    """將新的資料行寫入 Google Sheets。"""
    try:
        df = load_data(worksheet_name)
        new_df = pd.DataFrame([new_row_data])
        # 為了避免欄位順序錯亂，只保留工作表標題中存在的欄位
        valid_cols = [col for col in df.columns if col in new_df.columns]
        new_df = new_df[valid_cols]

        df = pd.concat([df, new_df], ignore_index=True)

        gsheets = gc.open_by_url(SHEET_URL)
        worksheet = gsheets.worksheet(worksheet_name)
        # 清空工作表並寫入新的 DataFrame (包含標題)
        worksheet.clear()
        set_with_dataframe(worksheet, df, include_index=False)
        return True, "資料已成功新增並存入 Google Sheets。"
    except Exception as e:
        return False, f"寫入資料到 {worksheet_name} 失敗: {e}"

def delete_data(worksheet_name, location_name):
    """根據地點名稱刪除 Google Sheets 中的資料行。"""
    if not location_name:
        return True, "未選擇地點，無需刪除。"

    try:
        df = load_data(worksheet_name)
        df_original_len = len(df)

        # 過濾掉與選擇地點名稱相符的行
        df_filtered = df[df['地點名稱'] != location_name]

        if len(df_filtered) == df_original_len:
            return True, f"找不到名稱為 '{location_name}' 的地點，地圖已刷新。"

        gsheets = gc.open_by_url(SHEET_URL)
        worksheet = gsheets.worksheet(worksheet_name)
        # 清空工作表並寫入新的 DataFrame (包含標題)
        worksheet.clear()
        set_with_dataframe(worksheet, df_filtered, include_index=False)
        return True, f"地點 '{location_name}' 已成功刪除並存入 Google Sheets。"
    except Exception as e:
        return False, f"刪除資料從 {worksheet_name} 失敗: {e}"

def create_map(df, map_title):
    """根據 DataFrame 創建 Folium 地圖。"""
    # 設定地圖的初始中心點和縮放級別
    center_lat, center_lng, zoom = 23.6978, 120.9605, 7 # 台灣中心點

    # 計算平均中心點，如果 DataFrame 不為空
    if not df.empty:
        # 計算經緯度的平均值作為中心點
        avg_lat = df['緯度'].mean()
        avg_lng = df['經度'].mean()
        if pd.notna(avg_lat) and pd.notna(avg_lng):
            center_lat, center_lng = avg_lat, avg_lng
            zoom = 10 # 有資料時放大一點

    m = folium.Map(location=(center_lat, center_lng), zoom_start=zoom, tiles="OpenStreetMap")

    for _, row in df.iterrows():
        lat = row['緯度']
        lng = row['經度']

        if pd.isna(lat) or pd.isna(lng):
            continue

        tooltip_str = row['地點名稱']

        if map_title == WORKSHEET_1_NAME:
            popup_html = f"<b>地點描述:</b> {row.get('地點描述', '無')}<br><b>飛航限制:</b> {row.get('飛航限制', '無')}"
            icon_name = "plane"
            icon_color = "red"
        else: # WORKSHEET_2_NAME
            popup_html = f"<b>個人備註:</b> {row.get('個人備註', '無')}"
            icon_name = "camera"
            icon_color = "blue"

        folium.Marker(
            location = [lat, lng],
            tooltip = tooltip_str,
            popup = folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(icon=icon_name, prefix='fa', color=icon_color),
        ).add_to(m)

    # 將 Folium 地圖渲染成 HTML 字串
    return m._repr_html_()

# 5. Gradio 介面互動函式

def update_map_and_dropdown(worksheet_name):
    """載入資料、生成地圖並更新刪除下拉選單選項。"""
    df = load_data(worksheet_name)
    map_html = create_map(df, worksheet_name)
    # 取得所有地點名稱作為下拉選單選項
    dropdown_choices = df['地點名稱'].tolist() if '地點名稱' in df.columns and not df.empty else []

    # *** 關鍵修正: 將 gr.Dropdown.update 替換為 gr.update ***
    return map_html, gr.update(choices=dropdown_choices)

# --- 空拍點位 (工作表 1) 相關操作 ---

def submit_drone_location(name, lat, lng, desc, limit):
    """新增空拍點位資料。"""
    try:
        lat = float(lat)
        lng = float(lng)
    except ValueError:
        # 如果轉換失敗，只更新地圖和下拉選單，並返回錯誤訊息
        return update_map_and_dropdown(WORKSHEET_1_NAME) + ("經緯度必須是數字！",)

    new_row = {
        '地點名稱': name,
        '經度': lng,
        '緯度': lat,
        '地點描述': desc,
        '飛航限制': limit
    }

    success, msg = write_data(WORKSHEET_1_NAME, new_row)

    # 重新整理地圖和下拉選單
    map_html, dropdown_update = update_map_and_dropdown(WORKSHEET_1_NAME)

    return map_html, dropdown_update, msg

def delete_drone_location(location_name):
    """刪除空拍點位資料。"""
    success, msg = delete_data(WORKSHEET_1_NAME, location_name)

    # 重新整理地圖和下拉選單
    map_html, dropdown_update = update_map_and_dropdown(WORKSHEET_1_NAME)

    return map_html, dropdown_update, msg

# --- 旅遊紀錄點 (工作表 2) 相關操作 ---

def submit_travel_record(name, lat, lng, note):
    """新增旅遊紀錄點資料。"""
    try:
        lat = float(lat)
        lng = float(lng)
    except ValueError:
        # 如果轉換失敗，只更新地圖和下拉選單，並返回錯誤訊息
        return update_map_and_dropdown(WORKSHEET_2_NAME) + ("經緯度必須是數字！",)

    new_row = {
        '地點名稱': name,
        '經度': lng,
        '緯度': lat,
        '個人備註': note
    }

    success, msg = write_data(WORKSHEET_2_NAME, new_row)

    # 重新整理地圖和下拉選單
    map_html, dropdown_update = update_map_and_dropdown(WORKSHEET_2_NAME)

    return map_html, dropdown_update, msg

def delete_travel_record(location_name):
    """刪除旅遊紀錄點資料。"""
    success, msg = delete_data(WORKSHEET_2_NAME, location_name)

    # 重新整理地圖和下拉選單
    map_html, dropdown_update = update_map_and_dropdown(WORKSHEET_2_NAME)

    return map_html, dropdown_update, msg


# 6. Gradio 介面定義
with gr.Blocks(title="地圖點位管理系統") as app:
    gr.Markdown("# 🗺️ Google Sheets 地圖點位管理系統")

    # 設置狀態變量來儲存訊息 (這個部分可省略，但保留無妨)
    status_msg_drone = gr.State(value="")
    status_msg_travel = gr.State(value="")

    with gr.Tab(WORKSHEET_1_NAME):
        # 初始化地圖和下拉選單
        initial_map_html_1, initial_choices_1 = update_map_and_dropdown(WORKSHEET_1_NAME)

        # 地圖區域
        map_output_drone = gr.HTML(initial_map_html_1, label=f"{WORKSHEET_1_NAME} 地圖")
        gr.Markdown("---")

        # 點位新增區域
        with gr.Accordion("➕ 新增空拍點位資料", open=True):
            with gr.Row():
                name_input_drone = gr.Textbox(label="地點名稱", placeholder="輸入地點名稱", scale=1)
                lat_input_drone = gr.Textbox(label="緯度 (lat)", placeholder="如: 25.0274", scale=1)
                lng_input_drone = gr.Textbox(label="經度 (lng)", placeholder="如: 121.5299", scale=1)

            desc_input_drone = gr.Textbox(label="地點描述", placeholder="如: 適合日出空拍", lines=2)
            limit_input_drone = gr.Textbox(label="飛航限制", placeholder="如: 禁飛區/限高20m", lines=1)
            submit_btn_drone = gr.Button("🚀 送出並更新地圖")

        # 點位刪除區域
        with gr.Accordion("➖ 刪除空拍點位資料", open=True):
            with gr.Row():
                # 注意: initial_choices_1 實際上是 gr.update 對象，但在 Blocks 結構中，
                # 我們必須先給予一個初始的 Dropdown 實例，這裡用 choices=initial_choices_1.kwargs['choices']
                # 或直接用 update_map_and_dropdown 的結果（但 Gradio 的做法通常是先用一個空的或靜態的列表）
                # 為了避免複雜，我們直接使用第一次 load_data 取得的 choices
                delete_dropdown_drone = gr.Dropdown(
                    label="選擇要刪除的地點名稱",
                    choices=load_data(WORKSHEET_1_NAME)['地點名稱'].tolist() if '地點名稱' in load_data(WORKSHEET_1_NAME).columns else [],
                    scale=2
                )
                delete_btn_drone = gr.Button("🗑️ 刪除並更新地圖", scale=1)

            # 狀態訊息顯示
            status_msg_display_drone = gr.Textbox(
                label="操作狀態/提示",
                value="",
                interactive=False,
                show_label=True,
                lines=1
            )

        # 綁定新增按鈕事件
        submit_btn_drone.click(
            submit_drone_location,
            inputs=[
                name_input_drone, lat_input_drone, lng_input_drone,
                desc_input_drone, limit_input_drone
            ],
            outputs=[map_output_drone, delete_dropdown_drone, status_msg_display_drone]
        )

        # 綁定刪除按鈕事件
        delete_btn_drone.click(
            delete_drone_location,
            inputs=[delete_dropdown_drone],
            outputs=[map_output_drone, delete_dropdown_drone, status_msg_display_drone]
        )


    with gr.Tab(WORKSHEET_2_NAME):
        # 初始化地圖和下拉選單
        initial_map_html_2, initial_choices_2 = update_map_and_dropdown(WORKSHEET_2_NAME)

        # 地圖區域
        map_output_travel = gr.HTML(initial_map_html_2, label=f"{WORKSHEET_2_NAME} 地圖")
        gr.Markdown("---")

        # 點位新增區域
        with gr.Accordion("➕ 新增旅遊紀錄點資料", open=True):
            with gr.Row():
                name_input_travel = gr.Textbox(label="地點名稱", placeholder="輸入地點名稱", scale=1)
                lat_input_travel = gr.Textbox(label="緯度 (lat)", placeholder="如: 25.0274", scale=1)
                lng_input_travel = gr.Textbox(label="經度 (lng)", placeholder="如: 121.5299", scale=1)

            note_input_travel = gr.Textbox(label="個人備註", placeholder="如: 排隊很久但值得", lines=2)
            submit_btn_travel = gr.Button("🚀 送出並更新地圖")

        # 點位刪除區域
        with gr.Accordion("➖ 刪除旅遊紀錄點資料", open=True):
            with gr.Row():
                # 初始下拉選單選項
                delete_dropdown_travel = gr.Dropdown(
                    label="選擇要刪除的地點名稱",
                    choices=load_data(WORKSHEET_2_NAME)['地點名稱'].tolist() if '地點名稱' in load_data(WORKSHEET_2_NAME).columns else [],
                    scale=2
                )
                delete_btn_travel = gr.Button("🗑️ 刪除並更新地圖", scale=1)

            # 狀態訊息顯示
            status_msg_display_travel = gr.Textbox(
                label="操作狀態/提示",
                value="",
                interactive=False,
                show_label=True,
                lines=1
            )

        # 綁定新增按鈕事件
        submit_btn_travel.click(
            submit_travel_record,
            inputs=[
                name_input_travel, lat_input_travel, lng_input_travel,
                note_input_travel
            ],
            outputs=[map_output_travel, delete_dropdown_travel, status_msg_display_travel]
        )

        # 綁定刪除按鈕事件
        delete_btn_travel.click(
            delete_travel_record,
            inputs=[delete_dropdown_travel],
            outputs=[map_output_travel, delete_dropdown_travel, status_msg_display_travel]
        )

# 7. 啟動 Gradio 介面
app.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4851c79625e0a636c3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipython-input-3082075908.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)
/tmp/ipython-input-3082075908.py:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_df], ignore_index=True)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://4851c79625e0a636c3.gradio.live
